-What about electrons? (I don't see any - 2 phys.me)

-1e11 and mDM?

-Compute T(E,z_in, z_dep) by doing a delta function injection

# Full Transfer Functions

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [168]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from astropy.io import fits

import numpy as np
from numpy.linalg import matrix_power
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_spectrum import nonrel_spec
from darkhistory.electrons.ics.ics_spectrum import rel_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import scattered_elec_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

from darkhistory.electrons import positronium as pos

from darkhistory.low_energy.lowE_deposition import compute_fs

# np.set_printoptions(threshold=np.nan)

In [3]:
soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
xe_std  = interp1d(soln[0,:], soln[2,:])
T_m_std = interp1d(soln[0,:], soln[1,:])

## Photon Transfer Functions

In [4]:
user = 'hongwan'

highengphot_tflist_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_photspec_60eV_complete_coarse.raw", "rb"))
lowengphot_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec_60eV_complete_coarse.raw", "rb"))
lowengelec_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec_60eV_complete_coarse.raw", "rb"))
CMB_engloss_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/CMB_engloss_60eV_complete_coarse.raw", "rb"))

In [5]:
photeng = highengphot_tflist_arr[0].eng
eleceng = lowengelec_tflist_arr[0].eng

#Split photeng into high and low energy. 
photeng_high = photeng[photeng > 60]
photeng_low  = photeng[photeng <= 60]

# Split eleceng into high and low energy. 
eleceng_high = eleceng[eleceng > 3000]
eleceng_low  = eleceng[eleceng <= 3000]


for highengphot_tflist in highengphot_tflist_arr:
    
#     highengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)

    for tf in highengphot_tflist:
        # tf.rebin(photeng_high)
#         tf.rebin(photeng)
        
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size, 0), (0, 0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    highengphot_tflist._eng = photeng
    highengphot_tflist._in_eng = photeng
    highengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in highengphot_tflist._tflist])
    )
    
# lowengphot_tflist.in_eng set to photeng_high
for lowengphot_tflist in lowengphot_tflist_arr:
    
#     lowengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengphot_tflist:
#         tf.rebin(photeng)
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        # Photons in the low energy bins should be immediately deposited.
        tf._grid_vals[0:photeng_low.size, 0:photeng_low.size] = np.identity(photeng_low.size)
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    lowengphot_tflist._eng = photeng
    lowengphot_tflist._in_eng = photeng
    lowengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengphot_tflist._tflist])
    )
    
# lowengelec_tflist.in_eng set to photeng_high 
for lowengelec_tflist in lowengelec_tflist_arr:
    
#     lowengelec_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengelec_tflist:
#         tf.rebin(eleceng)
        # Pad with zeros so that it becomes photeng x eleceng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = eleceng
        tf._rs = tf.rs[0]*np.ones_like(photeng)

    lowengelec_tflist._eng = eleceng
    lowengelec_tflist._in_eng = photeng
    lowengelec_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengelec_tflist._tflist])
    )
    
    for engloss in CMB_engloss_arr:
        engloss = np.pad(engloss, ((0,0),(photeng_low.size, 0)), 'constant')


In [6]:
xes = 0.5 + 0.5*np.tanh([-5., -4.1, -3.2, -2.3, -1.4, -0.5, 0.4, 1.3, 2.2, 3.1, 4])

highengphot_tf_interp = tflist.TransferFuncInterp(xes, highengphot_tflist_arr)
lowengphot_tf_interp = tflist.TransferFuncInterp(xes, lowengphot_tflist_arr)
lowengelec_tf_interp = tflist.TransferFuncInterp(xes, lowengelec_tflist_arr)


## Inverse Compton Scattering

Set up all of the inverse Compton scattering transfer functions (needs to be hidden away at some point). 

In [7]:
# %%prun

# Emax = 1e20
# Emin = 1e-8
# nEe = 5000
# nEp  = 5000

# dlnEp = np.log(Emax/Emin)/nEp
# lowengEp_rel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)        

# dlnEe = np.log(Emax/Emin)/nEe
# lowengEe_rel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

# Emax = 1e10
# Emin = 1e-8
# nEe = 5000
# nEp  = 5000

# dlnEp = np.log(Emax/Emin)/nEp
# lowengEp_nonrel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)  

# dlnEe = np.log(Emax/Emin)/nEe
# lowengEe_nonrel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

# print('********* Thomson regime scattered photon spectrum *********')
# ics_thomson_ref_tf = nonrel_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(400))
# print('********* Relativistic regime scattered photon spectrum *********')
# ics_rel_ref_tf = rel_spec(lowengEe_rel, lowengEp_rel, phys.TCMB(400), inf_upp_bound=True)
# print('********* Thomson regime energy loss spectrum *********')
# engloss_ref_tf = engloss_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(400), nonrel=True)
# print('********* Thomson regime scattered electron spectrum *********')
# scattered_elec_spec_nonrel_ref_tf = scattered_elec_spec(lowengEe_nonrel, lowengEe_nonrel, phys.TCMB(400), nonrel=True)
# print('********* Relativistic regime scattered electron spectrum *********')
# scattered_elec_spec_rel_ref_tf = scattered_elec_spec(lowengEe_rel, lowengEe_rel, phys.TCMB(400), nonrel=False)

In [8]:
# pickle.dump(ics_thomson_ref_tf, open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_thomson_ref_tf.p", "wb"))
# pickle.dump(ics_rel_ref_tf,     open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_rel_ref_tf.p", "wb"))
# pickle.dump(engloss_ref_tf,     open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/engloss_ref_tf.p", "wb"))
# pickle.dump(scattered_elec_spec_nonrel_ref_tf, open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/scattered_elec_spec_nonrel_ref_tf.p", "wb"))
# pickle.dump(scattered_elec_spec_rel_ref_tf, open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/scattered_elec_spec_rel_ref_tf.p", "wb"))


In [9]:
ics_thomson_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_thomson_ref_tf.p", "rb"))
ics_rel_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_rel_ref_tf.p", "rb"))
engloss_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/engloss_ref_tf.p", "rb"))
# scattered_elec_spec_nonrel_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/scattered_elec_spec_nonrel_ref_tf.p", "rb"))
# scattered_elec_spec_rel_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/scattered_elec_spec_rel_ref_tf.p", "rb"))


In [18]:
test = ics_rel_ref_tf_2[20]

In [169]:
%%prun
start_rs = 3000
rs = start_rs
while rs > 2500:
    (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
        ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
        eleceng, photeng, rs, fast=True
    )
    rs = np.exp(np.log(rs) - 0.002)
    print(rs)

2994.005996001995
2988.023968031968
2982.053892161797
2976.095744511174
2970.149501247497
2964.215138585785
2958.29263278858
2952.38196016585
2946.4830970748976
2940.5960199202623
2934.720705153627
2928.8571292737256
2923.0052688262463
2917.1651004037394
2911.336600645524
2905.5197462375927
2899.714513912521
2893.9208804493705
2888.1388226736008
2882.3683174569724
2876.609341717457
2870.8618724191438
2865.1258865721484
2859.4013612325193
2853.6882735021477
2847.9866005286754
2842.2963195054017
2836.6174076711964
2830.9498423104037
2825.293600752755
2819.648660373276
2814.0149985921985
2808.392592874867
2802.781420731652
2797.1814597178563
2791.5926874336296
2786.0150815238744
2780.4486196781604
2774.893279630632
2769.349039159922
2763.8158760890615
2758.29376828539
2752.7826936604692
2747.282630169994
2741.7935558137024
2736.3154486352896
2730.8482867223197
2725.3920482061376
2719.9467112617826
2714.512254107899
2709.0886550066516
2703.675892263636
2698.2739442277943
2692.8827892913273

## Evolution

In [12]:
# mDM = 605662.1322589782*2
# mDM = 10**9.104*2
# mDM = 1.27164e9*2
mDM = 10**7*2
# mDM = 10**6.5*2

phot_spec_init = spectools.rebin_N_arr(np.array([2]), 
                                      np.array([mDM/2]), 
                                      photeng)
phot_spec_init.switch_spec_type()

elec_spec_init = spectools.rebin_N_arr(np.array([2]), 
    np.array([mDM/2 - phys.me]),
    eleceng)

elec_spec_init.switch_spec_type()

In [13]:
print(phot_spec_init.toteng(bound_type = 'eng', bound_arr = np.array([1,1e13]))/mDM)

[1.]


In [170]:
%autoreload
phot_spec_init.rs = 3000
elec_spec_init.rs = 3000
end_rs   = 4
#mDM = 1e8
sigmav = 1e-26
tau = 1e25

def rate_func_swave_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)/(2*mDM)

def rate_func_swave_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)

def rate_func_decay_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('decay', rs, tau=tau)/mDM

def rate_func_decay_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('decay', rs, tau=tau)


def rate_func_swave_N_2(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)/(2*mDM)

def rate_func_swave_eng_2(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)


def evolve(
    in_spec_elec, in_spec_phot, 
    rate_func_N, rate_func_eng, xe_init, T_m_init,
    highengphot_tf_interp, lowengphot_tf_interp, 
    lowengelec_tf_interp, 
    end_rs, coarsen_factor=1, std_soln=False
):
    """
    Main function that computes the temperature and ionization history. 
    
    Parameters
    ----------
    in_spec_elec : Spectrum
        Spectrum per annihilation/decay into electrons. 
    in_spec_phot : Spectrum
        Spectrum per annihilation/decay into photons.
    rate_func_N : function
        Function describing the rate of annihilation/decay, dN/(dV dt)
    rate_func_eng : function
        Function describing the rate of annihilation/decay, dE/(dV dt)
    xe_init : float
        xe at the initial redshift. 
    T_m_init : float
        Matter temperature at the initial redshift. 
    highengphot_tf_interp : TransferFuncInterp
        Interpolation table for high energy photons. 
    lowengphot_tf_interp : TransferFuncInterp
        Interpolation table for low energy photons. 
    lowengelec_tf_interp : TransferFuncInterp
        Interpolation table for low energy electrons. 
    end_rs : float
        Final redshift to evolve to. 
    coarsen_factor : int
        Coarsening to apply to the transfer function matrix. 
    std_soln : bool
        If true, uses the standard TLA solution for f(z). 
    """
    
    # Initialize the next spectrum as None.
    next_highengphot_spec = None
    next_lowengphot_spec  = None
    next_lowengelec_spec  = None
    
    if (
        highengphot_tf_interp.dlnz    != lowengphot_tf_interp.dlnz
        or highengphot_tf_interp.dlnz != lowengelec_tf_interp.dlnz
        or lowengphot_tf_interp.dlnz  != lowengelec_tf_interp.dlnz
    ):
        raise TypeError('TransferFuncInterp objects must all have the same dlnz.')
    
    if in_spec_elec.rs != in_spec_phot.rs:
        raise TypeError('Input spectra must have the same rs.')
    
    # redshift/timestep related quantities. 
    dlnz = highengphot_tf_interp.dlnz
    prev_rs = None
    rs = in_spec_phot.rs
    dt = dlnz*coarsen_factor/phys.hubble(rs)
    
    # Function that changes the normalization from per annihilation to per baryon. 
    def norm_fac(rs): 
        return rate_func_N(rs) * dlnz * coarsen_factor /phys.hubble(rs) / (phys.nB * rs**3)
    
    # ICS for in_spec_elec goes here.
    
    elec_processes = False
    
    if in_spec_elec.totN() > 0:
        elec_processes = True
    
    if elec_processes:
        (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
            ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
            eleceng, photeng, rs, fast=True
        )

        ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec) * norm_fac(rs)
        ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec) * norm_fac(rs)
        positronium_phot_spec = pos.weighted_photon_spec(photeng)*in_spec_elec.totN()/2*norm_fac(rs)
        if positronium_phot_spec.spec_type != 'N':
            positronium_phot_spec.switch_spec_type()
        
#     print('input electron energy: ', (phys.me*2 + in_spec_elec.toteng())*norm_fac(rs))
#     print('ICS secondary photon spectrum energy: ', ics_phot_spec.toteng())
#     print('Upscattered photon energy: ', np.dot(in_spec_elec.N, continuum_loss)*norm_fac(rs))
#     print('ICS low energy electron spectrum energy: ', ics_lowengelec_spec.toteng() + phys.me*norm_fac(rs))
#     print('Positronium spectrum energy: ', positronium_phot_spec.toteng())
    
        init_inj_spec = in_spec_phot * norm_fac(rs) + ics_phot_spec + positronium_phot_spec
    else:
        init_inj_spec = in_spec_phot * norm_fac(rs)
    
    # Initialize the Spectra object that will contain all the 
    # output spectra during the evolution.
    out_highengphot_specs = Spectra([init_inj_spec], spec_type=init_inj_spec.spec_type)
    out_lowengphot_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    out_lowengelec_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    
    # Initialize the xe and T array that will store the solutions.
    xe_arr  = np.array([xe_init])
    T_m_arr = np.array([T_m_init])
    
    # Load the standard TLA solution if necessary. 
#     if std_soln:
    if True:
        soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
        xe_std  = interp1d(soln[0,:], soln[2,:])
        T_m_std = interp1d(soln[0,:], soln[1,:])

    # Define these methods for speed.
    append_highengphot_spec = out_highengphot_specs.append
    append_lowengphot_spec  = out_lowengphot_specs.append
    append_lowengelec_spec  = out_lowengelec_specs.append
    
    # For the tqdm bar
    pbar = tqdm(total = np.floor((np.log(rs) - np.log(end_rs))/dlnz/coarsen_factor).astype(int))
    
    # Loop while we are still at a redshift above end_rs.
    while rs > end_rs:
        
        # Update tqdm bar.
        pbar.update(1)
        
        # If prev_rs exists, calculate xe and T_m. 
        if prev_rs is not None:
            
            # f_continuum, f_lyman, f_ionH, f_ionHe, f_heat
            # f_raw takes in dE/(dV dt)
            if std_soln:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )  
            else:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
                
            std_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )
            
            backreaction_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
            
            f_raw = np.float64(f_raw)
            
            init_cond = np.array([T_m_arr[-1], xe_arr[-1], 0, 0])

            new_vals = tla.get_history(
                init_cond, f_raw[2], f_raw[1], f_raw[4], 
                rate_func_eng, np.array([prev_rs, rs]),
                reion_switch = False
            )

            T_m_arr = np.append(T_m_arr, new_vals[-1,0])
            xe_arr  = np.append(xe_arr,  new_vals[-1,1])
            
        print('x_e at '+str(rs)+': '+ str(xe_arr[-1]))
        print('Standard x_e at '+str(rs)+': '+str(xe_std(rs)))
        print('T_m at '+str(rs)+': '+ str(T_m_arr[-1]))
        print('Standard T_m at '+str(rs)+': '+str(T_m_std(rs)))
        if prev_rs is not None:
            print('Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', backreaction_f)
            print('Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', std_f)
            
        if std_soln:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_std(rs))
        else:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_arr[-1])
        
        if coarsen_factor > 1:
            prop_tf = np.zeros_like(highengphot_tf._grid_vals)
            for i in np.arange(coarsen_factor):
                prop_tf += matrix_power(highengphot_tf._grid_vals, i)
            lowengphot_tf._grid_vals = np.matmul(prop_tf, lowengphot_tf._grid_vals)
            lowengelec_tf._grid_vals = np.matmul(prop_tf, lowengelec_tf._grid_vals)
            highengphot_tf._grid_vals = matrix_power(
                highengphot_tf._grid_vals, coarsen_factor
            )
            
        next_highengphot_spec = highengphot_tf.sum_specs(out_highengphot_specs[-1])
        next_lowengphot_spec  = lowengphot_tf.sum_specs(out_highengphot_specs[-1])

        if elec_processes:
            next_lowengelec_spec  = lowengelec_tf.sum_specs(out_highengphot_specs[-1]) + ics_lowengelec_spec
        else:
            next_lowengelec_spec  = lowengelec_tf.sum_specs(out_highengphot_specs[-1])
        
        
        # Re-define existing variables.
        prev_rs = rs
        rs = np.exp(np.log(rs) - 0.002*coarsen_factor)
        
        dt = dlnz*coarsen_factor/phys.hubble(rs)
        next_highengphot_spec.rs = rs
        next_lowengphot_spec.rs  = rs
        next_lowengelec_spec.rs  = rs
        
        # Add the next injection spectrum to next_highengphot_spec
    
        if elec_processes:
            (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
                ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
                eleceng, photeng, rs, fast=True
            )

            ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec) * norm_fac(rs)
            ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec) * norm_fac(rs)
            positronium_phot_spec = pos.weighted_photon_spec(photeng)*in_spec_elec.totN()/2 * norm_fac(rs)
            if positronium_phot_spec.spec_type != 'N':
                positronium_phot_spec.switch_spec_type()
    
            next_inj_spec = in_spec_phot * norm_fac(rs) + ics_phot_spec + positronium_phot_spec
    
        else:
            next_inj_spec = in_spec_phot * norm_fac(rs)
        

        # This keeps the redshift. 
        next_highengphot_spec.N += next_inj_spec.N
        
        append_highengphot_spec(next_highengphot_spec)
        append_lowengphot_spec(next_lowengphot_spec)
        append_lowengelec_spec(next_lowengelec_spec)
        
        print('************************ END OF LOOP **************************')
        
        
    return (
        xe_arr, T_m_arr, 
        out_highengphot_specs, out_lowengphot_specs, out_lowengelec_specs
    )

In [174]:
%%prun
# a = evolve(
#         phot_spec_init*0, phot_spec_init, 
#         rate_func_decay_N, rate_func_decay_eng, .9999, phys.TCMB(phot_spec_init.rs),
#         highengphot_tf_interp, lowengphot_tf_interp, 
#         lowengelec_tf_interp, 
#         end_rs, coarsen_factor=1, std_soln=False
#     )

b_t = evolve(
        elec_spec_init, phot_spec_init*0, 
        rate_func_decay_N, rate_func_decay_eng, xe_std(phot_spec_init.rs), phys.TCMB(phot_spec_init.rs),
        #rate_func_decay_N, rate_func_decay_eng, 2e-4, phys.TCMB(phot_spec_init.rs),
        highengphot_tf_interp, lowengphot_tf_interp, 
        lowengelec_tf_interp, 
        2500, coarsen_factor=1, std_soln=True
    )

HBox(children=(IntProgress(value=0, max=91), HTML(value='')))

x_e at 3000: 0.999999999999
Standard x_e at 3000: 0.999999999999
T_m at 3000: 0.70459598745
Standard T_m at 3000: 0.70459598745
************************ END OF LOOP **************************


../darkhistory/spec/spectrum.py:758: RuntimeWarning: The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.
  warnings.warn("The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.", RuntimeWarning)
../darkhistory/spec/spectrum.py:663: RuntimeWarning: divide by zero encountered in log
  np.log(bound_arr),


x_e at 2994.005996001995: 0.9999999972104506
Standard x_e at 2994.005996001995: 0.9999999971389522
T_m at 2994.005996001995: 0.7031880249912993
Standard T_m at 2994.005996001995: 0.7031880248257363
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.002491431507677575, 0.016869527760967142, 0.11096420721947486, 8.034295311778317e-16, 0.8034295291320113]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.002491431507677575, 0.016869527760967142, 0.11096420721947486, 8.034353824834082e-16, 0.8034295291320113]
************************ END OF LOOP **************************
x_e at 2988.023968031968: 0.9999999971095765
Standard x_e at 2988.023968031968: 0.9999999970348642
T_m at 2988.023968031968: 0.7017830535651184
Standard T_m at 2988.023968031968: 0.7017830533974218
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.002644088124147089, 0.01759340426881339, 0.11235179111713758, 8.0730363169884e-16, 0.8072978956803625]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:

************************ END OF LOOP **************************
x_e at 2905.5197462375927: 0.9999999951857871
Standard x_e at 2905.5197462375927: 0.9999999950695925
T_m at 2905.5197462375927: 0.6824056658012247
Standard T_m at 2905.5197462375927: 0.6824056656120984
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0032177455074800703, 0.020239452742659775, 0.11787853890445543, 8.139677778037494e-16, 0.8139581688773573]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0032177455074800703, 0.020239452742659775, 0.11787853890445543, 8.139683811081313e-16, 0.8139581688773573]
************************ END OF LOOP **************************
x_e at 2899.714513912521: 0.9999999950036969
Standard x_e at 2899.714513912521: 0.9999999948841977
T_m at 2899.714513912521: 0.6810422175223313
Standard T_m at 2899.714513912521: 0.6810422173316614
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0032209149277532406, 0.020254596490903408, 0.11796341282241271, 8.140792766870602e-

************************ END OF LOOP **************************
x_e at 2825.293600752755: 0.9999999918449569
Standard x_e at 2825.293600752755: 0.9999999916727261
T_m at 2825.293600752755: 0.6635633189211451
Standard T_m at 2825.293600752755: 0.663563318709406
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0031985596571550636, 0.02020297874978574, 0.11866463917733043, 8.147471382480871e-16, 0.8147308819579657]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0031985596571550636, 0.02020297874978574, 0.11866463917733043, 8.147481274100515e-16, 0.8147308819579657]
************************ END OF LOOP **************************
x_e at 2819.648660373276: 0.9999999915269941
Standard x_e at 2819.648660373276: 0.9999999913497973
T_m at 2819.648660373276: 0.6622375176440156
Standard T_m at 2819.648660373276: 0.6622375174305744
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003194937146568, 0.020191940357003328, 0.11870757655642039, 8.147498791417899e-16, 0.81473

************************ END OF LOOP **************************
x_e at 2747.282630169994: 0.9999999859655271
Standard x_e at 2747.282630169994: 0.9999999857078479
T_m at 2747.282630169994: 0.6452412395554884
Standard T_m at 2747.282630169994: 0.6452412393187742
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003142743662327029, 0.020030955492204794, 0.11924232632060822, 8.143519293354228e-16, 0.814324034094703]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003142743662327029, 0.020030955492204794, 0.11924232632060822, 8.143535804634541e-16, 0.814324034094703]
************************ END OF LOOP **************************
x_e at 2741.7935558137024: 0.9999999854016408
Standard x_e at 2741.7935558137024: 0.9999999851363346
T_m at 2741.7935558137024: 0.6439520457836391
Standard T_m at 2741.7935558137024: 0.6439520455450423
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003138465175074395, 0.02001783480806063, 0.11928254312079653, 8.142945657840902e-16, 0.

************************ END OF LOOP **************************
x_e at 2671.425669892443: 0.9999999754522076
Standard x_e at 2671.425669892443: 0.9999999750620129
T_m at 2671.425669892443: 0.627425062250203
Standard T_m at 2671.425669892443: 0.6274250619857764
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0030820653110152987, 0.019840762366455683, 0.11979734317436347, 8.13318565520864e-16, 0.8132699603891871]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0030820653110152987, 0.019840762366455683, 0.11979734317436347, 8.133213782425282e-16, 0.8132699603891871]
************************ END OF LOOP **************************
x_e at 2666.0881578438775: 0.9999999744358422
Standard x_e at 2666.0881578438775: 0.9999999740338563
T_m at 2666.0881578438775: 0.6261714651892903
Standard T_m at 2666.0881578438775: 0.6261714649227774
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0030776603263012077, 0.01982671099083429, 0.11983638384305846, 8.132283988365674e-16,

************************ END OF LOOP **************************
x_e at 2597.6632441776474: 0.9999999563410596
Standard x_e at 2597.6632441776474: 0.9999999557421582
T_m at 2597.6632441776474: 0.6101008182041192
Standard T_m at 2597.6632441776474: 0.6101008179088636
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003032221527750743, 0.01969395099243028, 0.12043432046493346, 8.119610372776066e-16, 0.8118749668634859]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003032221527750743, 0.01969395099243028, 0.12043432046493346, 8.119659325673215e-16, 0.8118749668634859]
************************ END OF LOOP **************************
x_e at 2592.473109553961: 0.9999999544784934
Standard x_e at 2592.473109553961: 0.9999999538605272
T_m at 2592.473109553961: 0.6088818349694295
Standard T_m at 2592.473109553961: 0.6088818346718399
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0030306523137750797, 0.019691683549973676, 0.12049440939239807, 8.11858329675491e-16, 0

************************ END OF LOOP **************************
x_e at 2525.9375195055386: 0.999999921008301
Standard x_e at 2525.9375195055386: 0.9999999200753263
T_m at 2525.9375195055386: 0.5932549243166919
Standard T_m at 2525.9375195055386: 0.5932549239870998
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003009188641108983, 0.01965746826480537, 0.12127029744343587, 8.104432209345077e-16, 0.810288221820675]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003009188641108983, 0.01965746826480537, 0.12127029744343587, 8.1045193231763955e-16, 0.810288221820675]
************************ END OF LOOP **************************
x_e at 2520.890692975334: 0.9999999175356988
Standard x_e at 2520.890692975334: 0.9999999165720762
T_m at 2520.890692975334: 0.5920695991625007
Standard T_m at 2520.890692975334: 0.5920695988303092
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0030074668661003875, 0.01965453966497487, 0.12132980874789674, 8.103297039423634e-16, 0.8

In [16]:
plt.figure()
# plt.plot(a[3].rs, a[0])
plt.plot(b[3].rs, b[0])
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


plt.figure()
# plt.plot(a[3].rs, a[1]/phys.kB)
plt.plot(b[3].rs, b[1]/phys.kB)
plt.plot(b[3].rs, phys.TCMB(b[3].rs)/phys.kB, 'k:')
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


NameError: name 'b' is not defined

<Figure size 720x720 with 0 Axes>

In [ ]:
test_tf = highengphot_tf_interp.get_tf(4, xe_std(3000))
highengphot_tf_interp.xe

In [ ]:
plt.figure()

ax = plt.gca()

print(test_tf.in_eng[375])

test_tf.plot(ax, ind=250, indtype='ind')
test_tf.plot(ax, ind=377, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
test_spectra = highengphot_tf_interp._grid_vals[-1,-1,:,:]

plt.figure()
ax = plt.gca()

plt.plot(highengphot_tf_interp.eng, test_spectra[370, :])

ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
tf_before_interp = highengphot_tflist_arr[-1][-1]
tf_before_interp_2 = highengphot_tflist_arr[-1][0]

# print(tf_before_interp.in_eng[376])

plt.figure()
ax = plt.gca()
# tf_before_interp.plot(ax, ind=200, indtype='ind')
tf_before_interp.plot(ax, ind=370, indtype='ind')
# tf_before_interp_2.plot(ax, ind=376, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
lowengphot_spectra = b[3]
print(lowengphot_spectra.in_eng)

lowengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengphot_spectra[0].toteng())

plt.figure()
ax = plt.gca()

lowengphot_spectra.switch_spec_type()
lowengphot_spectra.plot(ax, ind=0, indtype='ind')
lowengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4, 1e3, 1e-10, 1e6])

In [ ]:
lowengelec_spectra = b[4]

lowengelec_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengelec_spectra[0].toteng())
utils.compare_arr([lowengelec_spectra[0].eng, lowengelec_spectra[0].dNdE])

plt.figure()
ax = plt.gca()

lowengelec_spectra.switch_spec_type()
lowengelec_spectra.plot(ax, ind=0, indtype='ind')
lowengelec_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1, 3e3, 1e-4, 1e3])

In [ ]:
highengphot_spectra = b[2]

highengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(2*10**9.104)
print(highengphot_spectra[0].toteng())
print(highengphot_spectra[1].toteng())

plt.figure()
ax = plt.gca()

highengphot_spectra.switch_spec_type()
highengphot_spectra.plot(ax, ind=1, indtype='ind')
highengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4,5e12, 1e-40, 1e10])